Task
1. Load and Explore the Data

Import the necessary libraries: pandas and sqlite3.
Connect to the IPL database and load the master table to understand the structure.
Load all the tables and print their column names to identify common columns.


In [5]:
import numpy as np
import pandas as pd
import sqlite3
conn = sqlite3.connect('/content/database.sqlite')

In [6]:
tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conn)
for table_name in tables['name']:
    try:
        columns = pd.read_sql_query(f"PRAGMA table_info({table_name});", conn)
        print(f"\nTable: {table_name}")
        print(columns['name'].tolist())
    except Exception as e:
        print(f"Error accessing table {table_name}: {e}")


Table: Player
['Player_Id', 'Player_Name', 'DOB', 'Batting_hand', 'Bowling_skill', 'Country_Name']

Table: Extra_Runs
['Match_Id', 'Over_Id', 'Ball_Id', 'Extra_Type_Id', 'Extra_Runs', 'Innings_No']

Table: Batsman_Scored
['Match_Id', 'Over_Id', 'Ball_Id', 'Runs_Scored', 'Innings_No']

Table: Batting_Style
['Batting_Id', 'Batting_hand']

Table: Bowling_Style
['Bowling_Id', 'Bowling_skill']

Table: Country
['Country_Id', 'Country_Name']

Table: Season
['Season_Id', 'Man_of_the_Series', 'Orange_Cap', 'Purple_Cap', 'Season_Year']

Table: City
['City_Id', 'City_Name', 'Country_id']

Table: Outcome
['Outcome_Id', 'Outcome_Type']

Table: Win_By
['Win_Id', 'Win_Type']

Table: Wicket_Taken
['Match_Id', 'Over_Id', 'Ball_Id', 'Player_Out', 'Kind_Out', 'Fielders', 'Innings_No']

Table: Venue
['Venue_Id', 'Venue_Name', 'City_Id']

Table: Extra_Type
['Extra_Id', 'Extra_Name']

Table: Out_Type
['Out_Id', 'Out_Name']

Table: Toss_Decision
['Toss_Id', 'Toss_Name']

Table: Umpire
['Umpire_Id', 'Umpire_

2. Query 1: Select All Columns from Player’s Table

Write and execute a SQL query to select all columns from the Player_Match table.


In [7]:
df = pd.read_sql_query("SELECT * FROM Player_Match;", conn)
print(df)

       Match_Id  Player_Id  Role_Id  Team_Id
0        335987          1        1        1
1        335987          2        3        1
2        335987          3        3        1
3        335987          4        3        1
4        335987          5        3        1
...         ...        ...      ...      ...
12689    981024        385        3       11
12690    981024        394        3       11
12691    981024        429        3       11
12692    981024        434        3        2
12693    981024        460        3       11

[12694 rows x 4 columns]


3. Query 2: Batsman vs Runs

Write and execute a SQL query to calculate the total runs scored by each batsman.


In [18]:
query1 = '''
SELECT p.Player_Name, SUM(b.Runs_Scored) AS Total_Runs
FROM Batsman_Scored b
JOIN Player_Match pm ON b.Match_Id = pm.Match_Id
JOIN Player p ON pm.Player_Id = p.Player_Id
GROUP BY p.Player_Name
ORDER BY Total_Runs DESC
'''
df = pd.read_sql_query(query1, conn)
print(df)

       Player_Name  Total_Runs
0         SK Raina       43101
1         MS Dhoni       41667
2        RG Sharma       41431
3          V Kohli       40563
4       KD Karthik       39424
..             ...         ...
464  NJ Rimmington         211
465     P Prasanth         183
466      DR Martyn         171
467       RV Pawar         117
468      SS Mundhe          51

[469 rows x 2 columns]


4. Query 3: Fifties and Hundreds

Write and execute a SQL query to calculate the number of fifties and hundreds scored by each batsman.


In [19]:
query2 = '''
SELECT
p.Player_Id,
p.Player_Name,
COUNT(CASE WHEN total_runs BETWEEN 50 AND 99 THEN 1 END) AS Fifties,
COUNT(CASE WHEN total_runs >= 100 THEN 1 END) AS Hundreds
FROM (
SELECT
b.Striker AS Player_Id,
SUM(bs.Runs_Scored) AS total_runs,
bs.Match_Id
FROM Batsman_Scored bs
JOIN Ball_by_Ball b ON bs.Match_Id = b.Match_Id
AND bs.Over_Id = b.Over_Id
AND bs.Ball_Id = b.Ball_Id
GROUP BY bs.Match_Id, b.Striker
) AS match_runs
JOIN Player p ON match_runs.Player_Id = p.Player_Id
GROUP BY p.Player_Id, p.Player_Name
ORDER BY Hundreds DESC, Fifties DESC
'''
df = pd.read_sql_query(query2, conn)
print(df)


     Player_Id      Player_Name  Fifties  Hundreds
0            8          V Kohli       29        32
1          187        DA Warner       21        28
2           40        G Gambhir       32        24
3          162         CH Gayle       19        24
4           85        AM Rahane       18        21
..         ...              ...      ...       ...
429         16         SB Joshi        0         0
430         15           Z Khan        0         0
431         13        AA Noffke        0         0
432         12          B Akhil        0         0
433          5  Mohammad Hafeez        0         0

[434 rows x 4 columns]


5. Query 4: Best Bowling Figures

Write and execute a SQL query to find the best bowling figures for each bowler.


In [22]:
query3 = '''
SELECT
p.Player_Id,
p.Player_Name,
best_bowling.Match_Id,
best_bowling.Wickets,
best_bowling.Runs_Conceded
FROM (
SELECT
b.Bowler AS Player_Id,
b.Match_Id,
COUNT(w.Player_Out) AS Wickets,
SUM(bs.Runs_Scored + IFNULL(er.Extra_Runs, 0)) AS Runs_Conceded
FROM Ball_by_Ball b
LEFT JOIN Wicket_Taken w
ON b.Match_Id = w.Match_Id
AND b.Over_Id = w.Over_Id
AND b.Ball_Id = w.Ball_Id
AND b.Bowler = w.Player_Out
LEFT JOIN Batsman_Scored bs
ON b.Match_Id = bs.Match_Id
AND b.Over_Id = bs.Over_Id
AND b.Ball_Id = bs.Ball_Id
LEFT JOIN Extra_Runs er
ON b.Match_Id = er.Match_Id
AND b.Over_Id = er.Over_Id
AND b.Ball_Id = er.Ball_Id
GROUP BY b.Bowler, b.Match_Id
) AS best_bowling
JOIN Player p ON best_bowling.Player_Id = p.Player_Id
WHERE best_bowling.Wickets > 0
ORDER BY best_bowling.Wickets DESC, best_bowling.Runs_Conceded ASC
'''
df = pd.read_sql_query(query3, conn)
print(df)

      Player_Id    Player_Name  Total_Runs  Fifties  Hundreds  Wickets  \
0            21       SK Raina       43101     44.0      19.0      2.0   
1            21       SK Raina       43101     44.0      19.0      2.0   
2            21       SK Raina       43101     44.0      19.0      2.0   
3            21       SK Raina       43101     44.0      19.0      2.0   
4            21       SK Raina       43101     44.0      19.0      2.0   
...         ...            ...         ...      ...       ...      ...   
6999        265  NJ Rimmington         211      0.0       0.0      0.0   
7000        447     P Prasanth         183      NaN       NaN      0.0   
7001        225      DR Martyn         171      0.0       0.0      NaN   
7002        467       RV Pawar         117      NaN       NaN      NaN   
7003        448      SS Mundhe          51      NaN       NaN      0.0   

      Runs_Conceded  
0              16.0  
1              24.0  
2              24.0  
3              27.0  
4

6. Query 5: Comprehensive Career Metrics

Combine all the previous chunks into a single comprehensive query to get detailed career metrics for players.

In [26]:
query4 = '''
WITH Runs AS (
SELECT p.Player_Id, p.Player_Name, SUM(b.Runs_Scored) AS Total_Runs
FROM Batsman_Scored b
JOIN Player_Match pm ON b.Match_Id = pm.Match_Id
JOIN Player p ON pm.Player_Id = p.Player_Id
GROUP BY p.Player_Id, p.Player_Name
),
FiftiesHundreds AS (
SELECT p.Player_Id,
COUNT(CASE WHEN total_runs BETWEEN 50 AND 99 THEN 1 END) AS Fifties,
COUNT(CASE WHEN total_runs >= 100 THEN 1 END) AS Hundreds
FROM (
SELECT
b.Striker AS Player_Id,
SUM(bs.Runs_Scored) AS total_runs
FROM Batsman_Scored bs
JOIN Ball_by_Ball b ON bs.Match_Id = b.Match_Id
AND bs.Over_Id = b.Over_Id
AND bs.Ball_Id = b.Ball_Id
GROUP BY bs.Match_Id, b.Striker
) AS match_runs
JOIN Player p ON match_runs.Player_Id = p.Player_Id
GROUP BY p.Player_Id
),
BestBowling AS (
SELECT
b.Bowler AS Player_Id,
COUNT(w.Player_Out) AS Wickets,
SUM(bs.Runs_Scored + IFNULL(er.Extra_Runs, 0)) AS Runs_Conceded,
b.Match_Id
FROM Ball_by_Ball b
LEFT JOIN Wicket_Taken w
ON b.Match_Id = w.Match_Id
AND b.Over_Id = w.Over_Id
AND b.Ball_Id = w.Ball_Id
AND b.Bowler = w.Player_Out
LEFT JOIN Batsman_Scored bs
ON b.Match_Id = bs.Match_Id
AND b.Over_Id = bs.Over_Id
AND b.Ball_Id = bs.Ball_Id
LEFT JOIN Extra_Runs er
ON b.Match_Id = er.Match_Id
AND b.Over_Id = er.Over_Id
AND b.Ball_Id = er.Ball_Id
GROUP BY b.Bowler, b.Match_Id
)
SELECT
p.Player_Id,
p.Player_Name,
r.Total_Runs,
fh.Fifties,
fh.Hundreds,
bw.Wickets,
bw.Runs_Conceded
FROM Player p
LEFT JOIN Runs r ON p.Player_Id = r.Player_Id
LEFT JOIN FiftiesHundreds fh ON p.Player_Id = fh.Player_Id
LEFT JOIN BestBowling bw ON p.Player_Id = bw.Player_Id
ORDER BY r.Total_Runs DESC, bw.Wickets DESC, bw.Runs_Conceded ASC;
'''
df = pd.read_sql_query(query4, conn)
print(df)

      Player_Id    Player_Name  Total_Runs  Fifties  Hundreds  Wickets  \
0            21       SK Raina       43101     44.0      19.0      2.0   
1            21       SK Raina       43101     44.0      19.0      2.0   
2            21       SK Raina       43101     44.0      19.0      2.0   
3            21       SK Raina       43101     44.0      19.0      2.0   
4            21       SK Raina       43101     44.0      19.0      2.0   
...         ...            ...         ...      ...       ...      ...   
6999        265  NJ Rimmington         211      0.0       0.0      0.0   
7000        447     P Prasanth         183      NaN       NaN      0.0   
7001        225      DR Martyn         171      0.0       0.0      NaN   
7002        467       RV Pawar         117      NaN       NaN      NaN   
7003        448      SS Mundhe          51      NaN       NaN      0.0   

      Runs_Conceded  
0              16.0  
1              24.0  
2              24.0  
3              27.0  
4